# Widzenie maszynowe
## Laboratorium 4 - Segmentacja
*Autor: Paweł Mendroch* - [Github](https://github.com/FrozenTear7/computer-vision-lab/tree/master/lab4)

Wykorzystując kod ze z [repozytorium](https://github.com/zhixuhao/unet), dokonuję segmentacji na przykładowych obrazach syntetycznych ze zbioru `hiragana`.

Poniżej przedstawiam kod wykorzystany do treningu i testów.

Przy użyciu biblioteki OpenCV dla języka Python, generuję maski dla obrazów syntetycznych znajdujących się w `/train/image` i zapisuję je do folderu `/train/label` trzymając się konwencji ustalonej przez oryginalnego autora.

Po odczytaniu obrazu część przezroczysta zostaje zamieniona w kolor czarny, który zamieniam na biały, a część obrazu, która stanowi faktyczny obraz (dłoń), koloruję na czarno, aby trzymać się kolorów narzuconych przez model.

Kod przygotowujący całość obrazów znajduje się w pliku `prepareImages.py`.

Przykładowe obrazy:

![Image](./results/train_image_example.png "Image")
![Label](./results/train_label_example.png "Label")


In [1]:
from model import *
from data import *
import os

Using TensorFlow backend.


Wykonuję przygotowanie danych na podstawie `dataPrepare.ipynb`.

In [2]:
data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')
myGenerator = trainGenerator(20, 'data/hiragana/train', 'image', 'label', data_gen_args, save_to_dir="data/hiragana/train/aug")

num_batch = 3
for i, batch in enumerate(myGenerator):
    if(i >= num_batch):
        break

Found 30 images belonging to 1 classes.
Found 30 images belonging to 1 classes.


Poniżej wykonuję trening sieci przy pomocy sieci Unet, jak podane w `trainUnet.ipynb`.
Metrykę dice coefficient dodałem w `model.py`.

```
def dice_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    return (2. * intersection + smooth) / (K.sum(K.square(y_true), -1) + K.sum(K.square(y_pred), -1) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)
```

In [5]:
data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')
myGene = trainGenerator(2, 'data/hiragana/train', 'image', 'label', data_gen_args, save_to_dir=None)

model = unet()
model_checkpoint = ModelCheckpoint('unet_hiragana.hdf5', monitor='loss', verbose=1, save_best_only=True)
model.fit_generator(myGene, steps_per_epoch=100, epochs=3, callbacks=[model_checkpoint])

Epoch 1/3
Found 30 images belonging to 1 classes.
Found 30 images belonging to 1 classes.
100/100 [==============================] - 57s 573ms/step - loss: 0.0228 - dice_coef: 0.9772 - acc: 0.9466

Epoch 00001: loss improved from inf to 0.02279, saving model to unet_hiragana.hdf5
Epoch 2/3
100/100 [==============================] - 47s 475ms/step - loss: 0.0036 - dice_coef: 0.9964 - acc: 0.9915

Epoch 00002: loss improved from 0.02279 to 0.00358, saving model to unet_hiragana.hdf5
Epoch 3/3
100/100 [==============================] - 47s 473ms/step - loss: 0.0027 - dice_coef: 0.9973 - acc: 0.9935

Epoch 00003: loss improved from 0.00358 to 0.00268, saving model to unet_hiragana.hdf5


In [6]:
testGene = testGenerator("data/hiragana/test")
model = unet()
model.load_weights("unet_hiragana.hdf5")
results = model.predict_generator(testGene, 30, verbose=1)
saveResult("data/hiragana/test", results)

30/30 [==============================] - 4s 140ms/step


Przeprowadziłem testy dla 3 zbiorów uczących o rozmiarach 30, 50 oraz 100 obrazów. Zbiór testowy zawierał 30 obrazów w każdym przypadku. Model uczyłem w 3 epokach po 100 kroków w każdej epoce i poniżej przedstawiam porównania wyników.

- train: 30

loss: 0.0026 - dice_coef: 0.9974 - acc: 0.9937

- train: 50

loss: 0.0026 - dice_coef: 0.9973 - acc: 0.9935

- train: 100

loss: 0.0026 - dice_coef: 0.9974 - acc: 0.9936

Jak widać na powyższych wynikach, większy zbiór treningowy niekoniecznie daje lepsze wyniki, dla wszystkich trzech przypadków wyniki były praktycznie identyczne.

Poniżej przedstawiam takie same testy przeprowadzone przy użyciu zdjęć RGB:

- train: 30

loss: 0.0027 - dice_coef: 0.9973 - acc: 0.9935

- train: 50

loss: 0.0026 - dice_coef: 0.9974 - acc: 0.9936

- train: 100

loss: 0.0026 - dice_coef: 0.9974 - acc: 0.9936

Wersja kolorowa nie wprowadziła zmian w rezultatach, wyniki są porównywalne do wersji w skali szarości.

Niestety nie udało mi się doprowadzić zapisu rezultatów do dobrze widocznych obrazów, co jest problemem powszechnym patrząc na zakładkę `Issues` na repo githubowym tego modelu.
Najpierw obraz wahał się między całym białym lub czarnym rezultatem, po różnych odnalezionych zmianach udało mi się doprowadzić do zapisu w szarym kolorze, na którym widać lekką poświatę obrazu wynikowego.
Obrazy wynikowe również zapisuję w formacie .tif, który nie jest obsługiwany przez Jupyter Notebook, przykładowy wynik znajduje się w folderze `results`.